# 列表
1. 列表的插入  
  list.insert(index, value)在指定index位置新增元素，开销较append大  
  list.append(value)在列表尾部新增元素
2. 列表的删除  
  list.pop(index)删除索引位置的元素  
  list.remove(value)删除列表中第一个值为value的元素
3. 列表的合并  
  list1 + list2的方式，生成一个新的列表，开销较大  
  list1.extend(list2)的方式，在原list1的基础上扩展list2，开销小
4. 列表的排序  
  list.sort(key)可以指定排序方式key，改变原列表顺序  
  sorted(list[,key])返回一个排序后的新列表  
5. 列表计数  
  list.count(value)返回一个value的计数  
6. enumerate，zip，reversed内置方法

# 字典

# 元组

# 函数

# itertools库  
1. combinations(iterable, k)根据k（可以理解为个数），返回iterable序列的组合。相当于排列组合的Cnm，不考虑元素的顺序  
2. permutations(iterable, k)功能类似combinations，只是要考虑元素的顺序，相当于Anm
3. product(*iterable,repeat=1)可以看做是repeat个数的嵌套for循环，获得序列的笛卡尔乘积。  
4. groupby(iterable[,keyfunc])根据每一个独立的key生成（key， sub-iterator)的元组,**如果是非有序序列，那么结果会不确定**   


In [1]:
from itertools import combinations, permutations, product, groupby

ls = list(range(1,5))
print(ls)
print(list(combinations(ls,2)))
print(list(permutations(ls,2)))
print(list(product(ls,repeat=2)))
ls1 = [k for k in range(5) if k%2]
ls1.extend(ls)

print(ls1)
for index, item in groupby(ls1):
    print(index, list(item), sep=":")
print(sorted(ls1))
for index, item in groupby(sorted(ls1)):
    print(index, list(item), sep=":")

[1, 2, 3, 4]
[(1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4)]
[(1, 2), (1, 3), (1, 4), (2, 1), (2, 3), (2, 4), (3, 1), (3, 2), (3, 4), (4, 1), (4, 2), (4, 3)]
[(1, 1), (1, 2), (1, 3), (1, 4), (2, 1), (2, 2), (2, 3), (2, 4), (3, 1), (3, 2), (3, 3), (3, 4), (4, 1), (4, 2), (4, 3), (4, 4)]
[1, 3, 1, 2, 3, 4]
1:[1]
3:[3]
1:[1]
2:[2]
3:[3]
4:[4]
[1, 1, 2, 3, 3, 4]
1:[1, 1]
2:[2]
3:[3, 3]
4:[4]


## Python socket编程相关  
selectors和select库，可以实现高效IO异步编程。selectors的defaultselector返回一个对象，具有register和unregister方法，注册回调函数，返回值是一个selectorkey对象，具有fd,fileobj, data等属性，一般用于socket等编程。selectors.Defaultselector.select方法返回一个（key,event）二元组，其中key是selectorkey对象，event是注册的事件，比如EVENT_READ。
HTTP 1.1版本connection默认是keep-alive，而1.0版本默认是closed  
python bounded function. python当调用实例方法时,会自动将实例传递给函数的第一个参数:

In [2]:
class Foo(object):
    pass
     
def func():
    pass   
         
if __name__ == "__main__":
    Foo.method = func
    f = Foo()
    print(Foo.method,f.method,func,sep="\n")

<function func at 0x7f9be4434a60>
<bound method func of <__main__.Foo object at 0x7f9be443abe0>>
<function func at 0x7f9be4434a60>


## Python functools库中的partial  
其中的patial本质上是一个装饰器  
```
def partial(func, *args, **kwargs):
    def wrapper(*ex_args, **ex_kwargs):
        print(args, ex_args, kwargs, ex_kwargs)
        new_kwargs = kwargs.copy()
        new_kwargs.update(ex_kwargs)
        return func(*(args+ex_args), **new_kwargs)
    return wrapper  
```  
用法就是先传入部分参数，后续调用在继续补全参数设置  


## Python asyncio库中对TCP、UDP、SSL等的封装  

asyncio库仅仅实现了对基本传输协议的封装，而更高层的比如HTTP有第三方库aiohttp  

这里记录几个函数：  
coroutine BaseEventLoop.create_connection(protocol_factory, host=None, port=None, *, ssl=None, family=0, proto=0, flags=0, sock=None, local_addr=None, server_hostname=None)  
此函数建立的是TCP连接，我们需要关注几个参数： host, Port就是通常的地址和端口， 而protocol_factory必须是一个返回Protocol对象的可调用对象, 而protocol对象是asyncio定义的TCP协议的流对象（此外还有DatagramProtocol和SubprocessProtocol ）  
Protocol实例里有connection_made（transport）和connection_lost（exc）函数，需要自己override.这两个函数分别在连接建立和连接断开时自动调用, 其中transport代表所建立的连接，需要根据需要进行保存。所有其他的方法必须在这两个方法之间进行调用， 例如Protocol.data_received(data)函数，当收到数据时会自动调用。  
在日常使用中可以使用 coroutine asyncio.open_connection(host=None, port=None, *, loop=None, limit=None, **kwds)， 他是对create_connection的封装，便于使用。  

同理还有TCP协议的create_server和start_server， 具体使用可以参照上述。

### Asyncio中的Task类  
Task本质上是Future的子类，它是对协程的进一步封装，一个事件循环同一时间只有一个task在运行， task不是线程安全。具有cancel(), exception()，done()，result()等方法。当使用asyncio.create_task时（py3.7支持），必须已经存在一个事件循环，否则会引发runtiimeerror，这是因为这一方法会自动将task注册到当前的事件循环（通过get_running_loop()方法获得）。通过asyncio.ensure_future创建的任务不会自动注册到事件循环，必须显式注册。当Task在运行过程中出现异常，task将不会再继续执行，调用task.exception()方法会得到异常信息（如果task运行正常，调用该方法会引发异常）。task的属性canceld会记录task是否被方法cancel()停止，当因为异常停止时canceld属性不会被设置。  
### 异步上下文管理器 async with  
异步上下文管理器指的是在enter和exit方法处能够暂停执行的上下文管理器。因此需要去实现两个魔方方法，__aenter__和__aexit__，这两个方法必须返回一个awaitable对象。而且async with必须在async def函数内部使用，否则会引发syntaxeroor  
  
```
async with EXPR as VAR:
    BLOCK
```

相当于以下代码：  
```
gr = (EXPR)
aexit = type(mgr).__aexit__
aenter = type(mgr).__aenter__(mgr)
exc = True
 
VAR = await aenter
try:
    BLOCK
except:
    if not await aexit(mgr, *sys.exc_info()):
        raise
else:
    await aexit(mgr, None, None, None)
```

